In [13]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

from pathlib import Path

In [24]:
import numpy as np
import soundfile as sf

# Load audio file
audio_path = 'audio.mp3'  # Replace 'path_to_your_audio_file.wav' with the actual path to your audio file
audio, sampling_rate = sf.read(audio_path)

# Convert audio array to a dictionary format
audio_dict = {
    'path': audio_path,
    'array': audio,
    'sampling_rate': sampling_rate
}

print(audio_dict)


{'path': 'audio.mp3', 'array': array([ 1.89960356e-05, -1.63675359e-05, -2.14778993e-06, ...,
       -5.85456219e-05, -3.70250200e-06,  3.67389584e-05]), 'sampling_rate': 48000}


In [25]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)
# model = model.to_bettertransformer()

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
# dataset[0]['audio']


# Pass audio input to the pipeline
result = pipe(audio_dict, generate_kwargs={"language": "turkish"})
print(result["text"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 Delikanlının adı Santiago idi. Sürüsüyle birlikte eski terk edilmiş kilisenin önüne geldiğinde güneş batmak üzereydi. Kilisenin çatısı çoktandır çökmüş. Bir zamanlar ayin eşyalarının kovunulduğu yerde kocaman bir firavun inciri büyümüştü. Delikanlı geceyi burada geçirmeye karar verdi. Bütün koyunlarını yıkık kapıdan içeri soktu. K koyunlarını yıkık kapıdan içeri soktu. Koyunların geceleyin kaçmalarına engel olacak şekilde kapıya birkaç tahta koydu. Bu bölgede kurt falan yoktu. Ama bir keresinde bir kaçak koyunu bulmak için ertesi gün bütün günde dolaşmak zorunda kalmıştı.


In [30]:
with open('result.txt', 'w', encoding='utf-8') as f:
    f.write(f'{result}')

{'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de',
 'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
        0.0005188 ]),
 'sampling_rate': 16000}